In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('..'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [46]:
!nvidia-smi

Wed Feb  5 13:23:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    63W / 149W |   8750MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pip install transformers

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import random
import pickle
import tensorflow.keras.backend as K
import gc
import time
import re
import os
import torch
from scipy.stats import spearmanr
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold,StratifiedKFold
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import AdamW,BertForSequenceClassification


In [0]:
sample_submission = pd.read_csv("/content/drive/My Drive/google-quest-challenge/sample_submission.csv")
test = pd.read_csv("/content/drive/My Drive/google-quest-challenge/test.csv")
train = pd.read_csv("/content/drive/My Drive/google-quest-challenge/train.csv")

MAX_SEQUENCE_LENGTH = 512

In [49]:
print('train shape =', train.shape)
print('test shape =', test.shape)

output_categories = list(train.columns[11:])
input_categories = list(train.columns[[1,2,5]])
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


In [50]:
len(output_categories)

30

In [51]:
output_categories[21:]

['answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [0]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [1] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input_q(title, question,max_sequence_length, 
                t_max_len=50, q_max_len=458):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    
    t_len = len(t)
    q_len = len(q)

    if (t_len+q_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            q_max_len = q_max_len + (t_max_len - t_len)
        else:
            t_new_len = t_max_len
      

        q_new_len = q_max_len
            
            
        if t_new_len+ q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
            
        if t_len > t_new_len:
            ind1 = floor(t_new_len/2)
            ind2 = ceil(t_new_len/2)
            t = t[:ind1]+t[-ind2:]
        else:
            t = t[:t_new_len]

        if q_len > q_new_len:
            ind1 = floor(q_new_len/2)
            ind2 = ceil(q_new_len/2)
            q = q[:ind1]+q[-ind2:]
        else:
            q = q[:q_new_len]

    
    return t, q


def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        if t_len > t_new_len:
            ind1 = floor(t_new_len/2)
            ind2 = ceil(t_new_len/2)
            t = t[:ind1]+t[-ind2:]
        else:
            t = t[:t_new_len]

        if q_len > q_new_len:
            ind1 = floor(q_new_len/2)
            ind2 = ceil(q_new_len/2)
            q = q[:ind1]+q[-ind2:]
        else:
            q = q[:q_new_len]

        if a_len > a_new_len:
            ind1 = floor(a_new_len/2)
            ind2 = ceil(a_new_len/2)
            a = a[:ind1]+a[-ind2:]
        else:
            a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ['<s>'] + title + ['</s>','</s>'] + question + ['</s>']

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q = _trim_input_q(t, q, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [0]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [0]:
import torch

def hard_sigmoid(x):
    """
    Computes element-wise hard sigmoid of x.
    See e.g. https://github.com/Theano/Theano/blob/master/theano/tensor/nnet/sigm.py#L279
    """
    x = (0.2 * x) + 0.5
    x = torch.clamp(x, 0.000001, 0.999999)
    return x

def postProcessing(x):
    x = torch.tensor(x)
    x = torch.cat((torch.sigmoid(x[:,:2]),hard_sigmoid(x[:,2].reshape(-1,1)),torch.sigmoid(x[:,3:11]),hard_sigmoid(x[:,11:13]),\
                        torch.sigmoid(x[:,13].reshape(-1,1)),hard_sigmoid(x[:,14:16]),torch.sigmoid(x[:,16:])),1)
    
    return x.numpy()

In [0]:
class TextDataset(torch.utils.data.TensorDataset):

    def __init__(self, x_train, idxs, targets=None):
        self.input_ids = x_train[0][idxs]
        self.input_masks = x_train[1][idxs]
        self.input_segments = x_train[2][idxs]
        self.targets = targets[idxs] if targets is not None else np.zeros((x_train[0].shape[0], 30))

    def __getitem__(self, idx):
#         x_train = self.x_train[idx]
        input_ids =  self.input_ids[idx]
        input_masks = self.input_masks[idx]
        input_segments = self.input_segments[idx]

        target = self.targets[idx]

        return input_ids, input_masks, input_segments, target

    def __len__(self):
        return len(self.input_ids)

In [0]:
from transformers import BertTokenizer,BertConfig,get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup,AutoTokenizer,AutoModel

In [0]:
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer, get_cosine_with_hard_restarts_schedule_with_warmup

In [0]:
pretrained_weights = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained('/content/drive/My Drive/RoBERTa')

In [59]:
output_categories

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [0]:
output_categories_q = output_categories[:21]

In [61]:
output_categories_q

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written']

In [63]:
x_train = compute_input_arays(train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
y_train = compute_output_arrays(train, output_categories)
x_test = compute_input_arays(test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

6079it [00:08, 733.69it/s]
476it [00:00, 551.80it/s]


In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertPreTrainedModel

ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP = {
    "roberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-pytorch_model.bin",
    "roberta-large": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-pytorch_model.bin",
    "roberta-large-mnli": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-mnli-pytorch_model.bin",
    "distilroberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-pytorch_model.bin",
    "roberta-base-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-openai-detector-pytorch_model.bin",
    "roberta-large-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-openai-detector-pytorch_model.bin",
}

class CustomizedRoberta(BertPreTrainedModel):
    config_class = RobertaConfig
    pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
    base_model_prefix = "roberta"

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.linear1 = nn.Linear(config.hidden_size*2, 21)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.RobertaConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import RobertaTokenizer, RobertaForSequenceClassification
        import torch

        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForSequenceClassification.from_pretrained('roberta-base')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

        """
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        avg_pool = torch.mean(outputs[0], 1)
        max_pool, _ = torch.max(outputs[0], 1)
        pooled_output = self.dropout(torch.cat((max_pool, avg_pool), 1))
        logits = self.linear1(pooled_output)


        outputs = (logits,) + outputs[2:]
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [0]:
import numpy as np

class callback:
    def __init__(self):
        self.score = list()
        self.model = list()
        self.data = list()
    
    def put(self, model,data, score):
        self.score.append(score)
        self.model.append(model)
        self.data.append(data)

    def get_model(self):
        ind = np.argmin(self.score)
        return self.model[ind]
    def get_data(self):
        ind = np.argmin(self.score)
        return self.data[ind]


In [0]:
with open("/content/drive/My Drive/qa_model/model0203_roberta_q.pkl","rb") as f:
    model_list = pickle.load(f)

In [67]:
len(model_list)

5

In [68]:
NFOLDS = 5
BATCH_SIZE = 4
EPOCHS = 4
SEED = 4354
num_warmup_steps = 100
lr = 3e-5


gradient_accumulation_steps = 1
seed_everything(SEED)


y_oof_q = np.zeros((len(train), 21))
test_pred_q = np.zeros((len(test), 21))

kf = KFold(n_splits=NFOLDS, shuffle=True)

test_loader = torch.utils.data.DataLoader(TextDataset(x_test, test.index),batch_size=BATCH_SIZE, shuffle=False)


for i, (train_idx, valid_idx) in enumerate(kf.split(x_train[0])):
    

    print(f'fold {i+1}')
    gc.collect()
    
    ## loader
    val_loader = torch.utils.data.DataLoader(TextDataset(x_train, valid_idx, y_train),batch_size=BATCH_SIZE, shuffle=False)
    

    net = model_list[i]
    net.cuda()


    valid_preds = np.zeros((len(valid_idx), 21))
        
    net.eval()
    for j,data in enumerate(val_loader):

        # get the inputs
        input_ids, input_masks, input_segments, labels = data
        pred = net(input_ids = input_ids.long().cuda(),
                        labels = None,
                        attention_mask = input_masks.cuda()
                        )[0]


        valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = pred.cpu().detach().numpy()

    y_oof_q[valid_idx] = valid_preds
    
    result = list()


    net.eval()
    with torch.no_grad():
        for data in test_loader:
            input_ids, input_masks, input_segments, labels = data
            y_pred = net(input_ids = input_ids.long().cuda(),
                                labels = None,
                                attention_mask = input_masks.cuda()
                            )[0]

            result.extend(y_pred.cpu().detach().numpy())
            
    test_pred_q += np.array(result)/NFOLDS


        


fold 1
fold 2
fold 3
fold 4
fold 5


In [0]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        if t_len > t_new_len:
            ind1 = floor(t_new_len/2)
            ind2 = ceil(t_new_len/2)
            t = t[:ind1]+t[-ind2:]
        else:
            t = t[:t_new_len]

        if q_len > q_new_len:
            ind1 = floor(q_new_len/2)
            ind2 = ceil(q_new_len/2)
            q = q[:ind1]+q[-ind2:]
        else:
            q = q[:q_new_len]

        if a_len > a_new_len:
            ind1 = floor(a_new_len/2)
            ind2 = ceil(a_new_len/2)
            a = a[:ind1]+a[-ind2:]
        else:
            a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q, a = _trim_input(t, q, a, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [70]:
x_train = compute_input_arays(train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
x_test = compute_input_arays(test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

6079it [00:15, 390.87it/s]
476it [00:01, 385.78it/s]


In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertPreTrainedModel

ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP = {
    "roberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-pytorch_model.bin",
    "roberta-large": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-pytorch_model.bin",
    "roberta-large-mnli": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-mnli-pytorch_model.bin",
    "distilroberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-pytorch_model.bin",
    "roberta-base-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-openai-detector-pytorch_model.bin",
    "roberta-large-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-openai-detector-pytorch_model.bin",
}

class CustomizedRoberta(BertPreTrainedModel):
    config_class = RobertaConfig
    pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
    base_model_prefix = "roberta"

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.linear1 = nn.Linear(config.hidden_size*2, 9)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.RobertaConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import RobertaTokenizer, RobertaForSequenceClassification
        import torch

        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForSequenceClassification.from_pretrained('roberta-base')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

        """
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        avg_pool = torch.mean(outputs[0], 1)
        max_pool, _ = torch.max(outputs[0], 1)
        pooled_output = self.dropout(torch.cat((max_pool, avg_pool), 1))
        logits = self.linear1(pooled_output)


        outputs = (logits,) + outputs[2:]
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [0]:
with open("/content/drive/My Drive/qa_model/model0203_roberta_a.pkl","rb") as f:
    model_list = pickle.load(f)

In [73]:
len(model_list)

5

In [74]:
NFOLDS = 5
BATCH_SIZE = 4
EPOCHS = 4
SEED = 4354
num_warmup_steps = 100
lr = 3e-5


gradient_accumulation_steps = 1
seed_everything(SEED)


y_oof_a = np.zeros((len(train), 9))
test_pred_a = np.zeros((len(test), 9))

kf = KFold(n_splits=NFOLDS, shuffle=True)

test_loader = torch.utils.data.DataLoader(TextDataset(x_test, test.index),batch_size=BATCH_SIZE, shuffle=False)


for i, (train_idx, valid_idx) in enumerate(kf.split(x_train[0])):
    

    print(f'fold {i+1}')
    gc.collect()
    
    ## loader
    val_loader = torch.utils.data.DataLoader(TextDataset(x_train, valid_idx, y_train),batch_size=BATCH_SIZE, shuffle=False)
    

    net = model_list[i]
    net.cuda()


    valid_preds = np.zeros((len(valid_idx), 9))
        
    net.eval()
    for j,data in enumerate(val_loader):

        # get the inputs
        input_ids, input_masks, input_segments, labels = data
        pred = net(input_ids = input_ids.long().cuda(),
                        labels = None,
                        attention_mask = input_masks.cuda()
                        )[0]


        valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = pred.cpu().detach().numpy()

    y_oof_a[valid_idx] = valid_preds
    
    result = list()


    net.eval()
    with torch.no_grad():
        for data in test_loader:
            input_ids, input_masks, input_segments, labels = data
            y_pred = net(input_ids = input_ids.long().cuda(),
                                labels = None,
                                attention_mask = input_masks.cuda()
                            )[0]

            result.extend(y_pred.cpu().detach().numpy())
            
    test_pred_a += np.array(result)/NFOLDS


        


fold 1
fold 2
fold 3
fold 4
fold 5


In [0]:
y_oof = np.concatenate((y_oof_q,y_oof_a), axis=1)

In [0]:
test_pred = np.concatenate((test_pred_q,test_pred_a), axis=1)

In [0]:
with open("/content/drive/My Drive/qa_model/y_oof_roberta.pkl","wb") as f:
    pickle.dump(y_oof,f)

In [0]:
test_pred.shape

In [0]:
y_oof.shape

In [0]:
def postProcessing(x):

    def f_0(x):
        if x <= 0.1666:
            x = 0.0000001
        if x >= 0.833:
            x = 0.99999999
        return x

    def f_1(x):
        if x <= 0.1666:
            x = 0.00000001
        # if x >= 0.8333:
        #     x = 0.99999999
        return x

    x[:,2] = np.array(list(map(f_0,x[:,2])))
    x[:,5] = np.array(list(map(f_0,x[:,5])))
    x[:,11] = np.array(list(map(f_0,x[:,11])))
    x[:,15] = np.array(list(map(f_0,x[:,15])))

    x[:,8] = np.array(list(map(f_1,x[:,8])))
    x[:,12] = np.array(list(map(f_1,x[:,12])))
    x[:,14] = np.array(list(map(f_1,x[:,14])))

    return x

In [0]:
def f_0(x):
        if x <= 0.1666:
            x = 0.0000001
        if x >= 0.833:
            x = 0.99999999
        return x

In [15]:
a = torch.sigmoid(torch.tensor(y_oof)).numpy()

NameError: ignored

In [0]:
a[:,13] = np.array(list(map(f_0,a[:,13])))

In [0]:
a = hard_sigmoid(torch.tensor(y_oof))

In [0]:
spearmanr(y_train[:, 13], a[:, 13]).correlation

0.20255582301617409

In [0]:
spearmanr(y_train[:, 13], a[:, 13]).correlation

0.20980806104569558

In [0]:
spearmanr(y_train[:, 13], a[:, 13]).correlation

0.17676696307738426

In [0]:
a

tensor([[ 3.2326,  1.4640, -1.2751,  ..., -1.7468,  1.3681,  2.1423],
        [ 3.1399,  1.6450, -3.9002,  ..., -5.3079, -0.6503,  2.3483],
        [ 2.1319, -0.5572, -5.0416,  ..., -0.8787,  1.4793,  2.3981],
        ...,
        [ 2.1425,  0.6928, -5.3844,  ..., -1.8835,  0.3801,  1.2380],
        [ 2.2436,  0.7396,  0.8859,  ..., -3.3095,  1.9918,  2.8196],
        [ 3.8875,  1.5113, -2.0171,  ..., -4.0199,  2.1642,  2.3808]],
       dtype=torch.float64)

In [0]:
a = postProcessing(a)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [0]:
oof_score = 0
for i in range(30):
    oof_score += np.nan_to_num(
            spearmanr(y_train[:, i], y_oof[:, i]).correlation / 30)

In [79]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.40207806909928145


In [0]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.4272448191760734


In [0]:
print("Out of folds score = {}",oof_score)

Out of folds score = {} 0.4020780273694078


In [0]:
sample_submission.loc[:, output_categories] = test_pred
sample_submission.to_csv('/content/drive/My Drive/qa_submission/submission0203.csv', index=False)